In [4]:
from google.colab import files
uploaded = files.upload()

Saving test_data.xlsx to test_data.xlsx
Saving train_data.xlsx to train_data.xlsx


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

In [50]:
class LoanDefaultPrediction:
    def __init__(self, model_type='logistic_regression'):
        self.model_type = model_type
        self.model = None

    def load(self, data):
        """
        Load the dataset and separate features and target variable
        """
        self.df = data
        self.X = self.df.drop('loan_status', axis=1)
        self.y = self.df['loan_status']

    def preprocess(self):
        """
        Apply preprocessing steps like scaling numeric features and encoding categorical features
        """
        # Handle datetime columns, if any
        date_columns = self.X.select_dtypes(include=['datetime64']).columns
        for col in date_columns:
            self.X[col] = pd.to_datetime(self.X[col])
            self.X[f'{col}_year'] = self.X[col].dt.year
            self.X[f'{col}_month'] = self.X[col].dt.month
            self.X[f'{col}_day'] = self.X[col].dt.day
            self.X[f'{col}_weekday'] = self.X[col].dt.weekday
            self.X = self.X.drop([col], axis=1)

        # Encode categorical columns (example with 'home_ownership' and 'purpose')
        if 'home_ownership' in self.X.columns:
            self.X['home_ownership'] = self.X['home_ownership'].map({'OWN': 0, 'MORTGAGE': 1, 'RENT': 2})

        if 'purpose' in self.X.columns:
            self.X['purpose'] = self.X['purpose'].map({'debt_consolidation': 0, 'credit_card': 1, 'other': 2, 'home_improvement': 3})

        # Ensure there are no missing values - ONLY for numeric columns
        numeric_columns = self.X.select_dtypes(include=['number']).columns
        self.X[numeric_columns] = self.X[numeric_columns].fillna(self.X[numeric_columns].median())  # Or self.X.dropna() if you want to drop missing rows

        # Apply StandardScaler to numerical features
        scaler = StandardScaler()
        numeric_columns = ['annual_inc', 'loan_amnt', 'cibil_score']
        self.X[numeric_columns] = scaler.fit_transform(self.X[numeric_columns])

        # Check final data types
        print(self.X.dtypes)

    def train(self):
        """
        Train the model using a split of training and testing data
        """
        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)

        # Choose the model
        if self.model_type == 'logistic_regression':
            self.model = LogisticRegression(max_iter=1000)
        elif self.model_type == 'random_forest':
            self.model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

        # Train the model
        self.model.fit(X_train, y_train)

    def test(self):
        """
        Test the model and print evaluation metrics
        """
        y_pred = self.model.predict(self.X)
        print("Accuracy:", accuracy_score(self.y, y_pred))
        print("Classification Report:\n", classification_report(self.y, y_pred))

    def predict(self, X):
        """
        Predict loan default for new data
        """
        return self.model.predict(X)

In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Load the dataset
train_df = pd.read_excel('train_data.xlsx')

# Handle missing values
# Separate numeric and categorical columns
numeric_columns = train_df.select_dtypes(include=['number']).columns
categorical_columns = train_df.select_dtypes(exclude=['number']).columns

# Fill missing values for numeric columns with the median
train_df[numeric_columns] = train_df[numeric_columns].fillna(train_df[numeric_columns].median())

# For categorical columns, fill missing values with the mode (most frequent value)
for col in categorical_columns:
    train_df[col] = train_df[col].fillna(train_df[col].mode()[0])

# Check if there are any remaining missing values
print(train_df.isnull().sum())

# Handle datetime columns - Transform or drop them
train_df['transaction_date'] = pd.to_datetime(train_df['transaction_date'], errors='coerce')  # Convert to datetime

# Extract date-related features
train_df['transaction_year'] = train_df['transaction_date'].dt.year
train_df['transaction_month'] = train_df['transaction_date'].dt.month
train_df['transaction_day'] = train_df['transaction_date'].dt.day
train_df['transaction_weekday'] = train_df['transaction_date'].dt.weekday

# Drop original datetime column
train_df.drop('transaction_date', axis=1, inplace=True)

# Encode categorical columns (example with 'home_ownership', 'purpose', etc.)
train_df['home_ownership'] = train_df['home_ownership'].map({'OWN': 0, 'MORTGAGE': 1, 'RENT': 2})
train_df['purpose'] = train_df['purpose'].map({'debt_consolidation': 0, 'credit_card': 1, 'other': 2, 'home_improvement': 3})

# Check if there are any columns with string labels that need encoding
# Apply Label Encoding to other categorical columns with string labels
le = LabelEncoder()

# List of columns to encode using LabelEncoder (e.g. sub_grade, application_type, verification_status)
label_encode_cols = ['sub_grade', 'application_type', 'verification_status']

for col in label_encode_cols:
    train_df[col] = le.fit_transform(train_df[col])

# Handle 'term' column (convert string '36 months' to integer 36)
train_df['term'] = train_df['term'].str.extract('(\d+)').astype(int)

# Check the data types again after encoding
print(train_df.dtypes)

# Separate features and target variable
X = train_df.drop('loan_status', axis=1)
y = train_df['loan_status']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Check for missing values in train and test sets
print("Missing values in X_train:", X_train.isnull().sum().sum())
print("Missing values in X_test:", X_test.isnull().sum().sum())

# Handle missing values explicitly using SimpleImputer if there are any NaNs
imputer = SimpleImputer(strategy='median')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Apply StandardScaler to numeric features
scaler = StandardScaler()

# Keep track of the columns in X_train
X_train_columns = X_train.columns if isinstance(X_train, pd.DataFrame) else X.columns

# Scale the data and convert back to DataFrame with column names
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train_columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train_columns)

# Check for any remaining NaNs after transformation
print("Missing values after imputation and scaling in X_train:", X_train.isnull().sum().sum())
print("Missing values after imputation and scaling in X_test:", X_test.isnull().sum().sum())

# Function to train and evaluate models
def train_and_evaluate(model, X_train, X_test, y_train, y_test):
    """
    Trains a given model, evaluates its performance, and returns evaluation metrics.
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print(f"Model: {model.__class__.__name__}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

# Initialize models
log_reg_model = LogisticRegression(max_iter=1000)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
svm_model = SVC()

# Train and evaluate each model
print("Logistic Regression Model")
train_and_evaluate(log_reg_model, X_train, X_test, y_train, y_test)

print("\nRandom Forest Model")
train_and_evaluate(rf_model, X_train, X_test, y_train, y_test)

print("\nSupport Vector Machine Model")
train_and_evaluate(svm_model, X_train, X_test, y_train, y_test)


customer_id            0
transaction_date       0
sub_grade              0
term                   0
home_ownership         0
cibil_score            0
total_no_of_acc        0
annual_inc             0
int_rate               0
purpose                0
loan_amnt              0
application_type       0
installment            0
verification_status    0
account_bal            0
emp_length             0
loan_status            0
dtype: int64
customer_id              int64
sub_grade                int64
term                     int64
home_ownership           int64
cibil_score              int64
total_no_of_acc          int64
annual_inc             float64
int_rate               float64
purpose                float64
loan_amnt                int64
application_type         int64
installment            float64
verification_status      int64
account_bal              int64
emp_length               int64
loan_status              int64
transaction_year         int32
transaction_month        int32
tran

Logistic Regression Model
Model: LogisticRegression
Accuracy: 0.7638367729831145
Confusion Matrix:
 [[ 2247  6664]
 [ 1392 23809]]
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.25      0.36      8911
           1       0.78      0.94      0.86     25201

    accuracy                           0.76     34112
   macro avg       0.70      0.60      0.61     34112

weighted avg       0.74      0.76      0.73     34112

--------------------------------------------------------------------------


Random Forest Model
Model: RandomForestClassifier
Accuracy: 0.7645403377110694
Confusion Matrix:
 [[ 2691  6220]
 [ 1812 23389]]
Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.30      0.40      8911
           1       0.79      0.93      0.85     25201

    accuracy                           0.76     34112
   macro avg       0.69      0.62      0.63     34112

weighted avg       0.74      0.76      0.74     34112

--------------------------------------------------------------------------

Support Vector Machine Model
Model: SVC
Accuracy: 0.7661819887429644
Confusion Matrix:
 [[ 2128  6783]
 [ 1193 24008]]
Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.24      0.35      8911
           1       0.78      0.95      0.86     25201

    accuracy                           0.77     34112
   macro avg       0.71      0.60      0.60     34112

weighted avg       0.74      0.77      0.72     34112
